#### Saving Bayestar 2017 with sigma

In [1]:
import healpy as hp
from dustmaps.bayestar import BayestarQuery
from astropy.coordinates import SkyCoord
import astropy.units as u
import numpy as np

In [2]:
b17= BayestarQuery(version='bayestar2017')

Loading pixel_info ...
Loading samples ...
Loading best_fit ...
Replacing NaNs in reliable distance estimates ...
Sorting pixel_info ...
Extracting hp_idx_sorted and data_idx at each nside ...
  nside = 64
  nside = 128
  nside = 256
  nside = 512
  nside = 1024
t = 80.044 s
  pix_info:   0.898 s
   samples:  66.199 s
      best:   3.799 s
       nan:   0.090 s
      sort:   8.851 s
       idx:   0.208 s


In [3]:
b17map = np.zeros(hp.pixelfunc.nside2npix(2048))

In [4]:
allpix2048 = np.arange(hp.pixelfunc.nside2npix(2048))

In [5]:
lb2048 = hp.pixelfunc.pix2ang(2048, allpix2048, lonlat=True)

In [6]:
lb2048

(array([ 45., 135., 225., ..., 135., 225., 315.]),
 array([ 89.97715732,  89.97715732,  89.97715732, ..., -89.97715732,
        -89.97715732, -89.97715732]))

In [7]:
#print('Max Dist', b17.distances[-1], b19.distances[-1])

In [8]:
mindist = b17.distances[-1]
print(mindist)

63.095726013183594 kpc


In [9]:
coords2048 = SkyCoord(l=lb2048[0]*u.deg, b=lb2048[1]*u.deg, distance = mindist, frame='galactic')

In [10]:
b17map = b17(coords2048, mode='mean')

In [12]:
pct = [16, 84]
bpct = b17(coords2048, mode='percentile', pct=pct)

/n/home02/nmudur/.conda/envs/pytorch_env2/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1584: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanquantile_1d, axis, a, q,


In [13]:
bpct.shape

(50331648, 2)

In [14]:
bsigma_map = (bpct[...,1] - bpct[...,0])/2

In [15]:
assert len(bsigma_map)==hp.nside2npix(2048)==len(b17map)

In [18]:
'''bsigma_map_check2 = (bpct[:, 1] - bpct[:, 0])/2
np.all((bsigma_map == bsigma_map_check2)[~np.isnan(bsigma_map)]) #same value wherever not nan'''

True

In [19]:
import h5py
with h5py.File('../reference_maps/bayestar2017_wsigma.hdf5', 'w') as f:
    dset = f.create_dataset("mean", data = b17map)
    dset = f.create_dataset("sigma", data = bsigma_map)
    dset = f.create_dataset("coords_lb", data = np.vstack(lb2048).T)

In [14]:
del b17map

##### 2019

In [2]:
import healpy as hp
from dustmaps.bayestar import BayestarQuery
from astropy.coordinates import SkyCoord
import astropy.units as u
import numpy as np
import gc
import h5py

b19 = BayestarQuery(version='bayestar2019')
mindist = b19.distances[-1]
print(mindist)

b19map = np.zeros(hp.pixelfunc.nside2npix(2048))
allpix2048 = np.arange(hp.pixelfunc.nside2npix(2048))
lb2048 = hp.pixelfunc.pix2ang(2048, allpix2048, lonlat=True)
coords2048 = SkyCoord(l=lb2048[0]*u.deg, b=lb2048[1]*u.deg, distance = mindist, frame='galactic')
b19map = b19(coords2048, mode='mean')
del b19
gc.collect()
with h5py.File('../reference_maps/bayestar2019.hdf5', 'w') as f:
    dset = f.create_dataset("mean", data = b19map)
    dset = f.create_dataset("coords_lb", data = np.vstack(lb2048).T)

Loading pixel_info ...
Loading samples ...
Loading best_fit ...
Replacing NaNs in reliable distance estimates ...
Sorting pixel_info ...
Extracting hp_idx_sorted and data_idx at each nside ...
  nside = 64
  nside = 128
  nside = 256
  nside = 512
  nside = 1024
t = 41.586 s
  pix_info:   0.596 s
   samples:  26.292 s
      best:   4.285 s
       nan:   0.075 s
      sort:  10.167 s
       idx:   0.170 s
59.56621435290109 kpc
